# EquiLend D&A Daily Digest

In [9]:
import datetime
import docx
import pandas as pd
import oracledb
import smtplib
import subprocess
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from dotenv import load_dotenv

# --- 1. Configuration & Setup ---
# Load environment variables from .env file
load_dotenv()

# Local save directory (create if not exists)
save_dir = os.path.expanduser('~/equilend_daily')
os.makedirs(save_dir, exist_ok=True)

# Cloud sync remotes (from environment or default)
cloud_remotes_str = os.getenv('CLOUD_REMOTES', 'google_drive:daily_digest/,onedrive:equilend_daily/')
cloud_remotes = [remote.strip() for remote in cloud_remotes_str.split(',') if remote.strip()]

# Word document path
doc_path = os.getenv('WORD_DOC_PATH', 'daily_content.docx')

# --- 2. Data Extraction from Word ---
def extract_section(doc, start_header, end_header=None):
    """Extracts content between two headers in a Word document."""
    content = []
    capturing = False
    for para in doc.paragraphs:
        text = para.text.strip()
        if text == start_header:
            capturing = True
            continue
        if end_header and text == end_header:
            break
        if capturing and text:
            content.append(text)
    return content

def extract_headlines(doc, start_header, end_header):
    """Extracts structured headlines from the document."""
    headlines = []
    title = ''
    desc = ''
    data_shows = ''
    in_desc = False
    
    headlines_section = extract_section(doc, start_header, end_header)
    for para in headlines_section:
        if para.isupper() and not para.startswith('What Our Data Shows:'):
            if title:
                headlines.append({'title': title, 'description': desc.strip(), 'data_shows': data_shows.replace('What Our Data Shows: ', '')})
            title = para
            desc, data_shows = '', ''
            in_desc = True
        elif para.startswith('What Our Data Shows:'):
            data_shows = para
            in_desc = False
        elif in_desc:
            desc += ' ' + para
            
    if title: # Append the last one
        headlines.append({'title': title, 'description': desc.strip(), 'data_shows': data_shows.replace('What Our Data Shows: ', '')})
    return headlines

try:
    doc = docx.Document(doc_path)
    market_notes = extract_section(doc, '📈 Market Notes', '🔥 Major Headlines & What Our Data Shows')
    headlines = extract_headlines(doc, '🔥 Major Headlines & What Our Data Shows', '📊 Today\'s Specials & Hard-to-Borrow')
    key_takeaways = extract_section(doc, '💡 Key Takeaways', 'Source: EquiLend Data & Analytics')
except Exception as e:
    print(f"Error reading Word document '{doc_path}': {e}")
    # Add fallback or exit if essential
    market_notes, headlines, key_takeaways = [], [], []

# --- 3. Database Interaction ---
def get_db_connection():
    """Establishes and returns an Oracle database connection."""
    try:
        user = os.getenv('ORACLE_DB_USER')
        password = os.getenv('ORACLE_DB_PASSWORD')
        dsn = os.getenv('ORACLE_DB_DSN')
        
        if not all([user, password, dsn]):
            print("Error: Database credentials not found in .env file.")
            return None
            
        return oracledb.connect(user=user, password=password, dsn=dsn)
    except Exception as e:
        print(f"Database connection failed: {e}")
        return None

def fetch_data(conn):
    """Fetches and returns data from the Oracle database."""
    today = datetime.date.today()
    from_date = (today - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
    to_date = (today - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

    sql = f"""
    SELECT 
        r.business_date, s.ticker, s.security_description AS name,
        a.shares * a.security_price_c AS market_cap, a.shares AS sharesoutstanding,
        c.country_name, i.sector_name, i.industry_name AS industry,
        d.return_loan_qty_amt, d.new_loan_qty_amt, a.security_price_c AS price,
        d.loan_qty_amt_c, a.short_squeeze_score_c AS short_squeeze_score,
        d.avg_spread_all_amt_c AS fee_bps, a.short_interest_c AS short_interest,
        a.days_to_cover_c AS days_to_cover, d.active_utilization_c AS active_utilization,
        d.loan_val_amt_c AS on_loan_value, d.bb_val_amt_c,
        d.bb_qty_amt_c AS borrower_count, d.active_lendable_qty_c, d.active_lendable_val_c
    FROM 
        (SELECT MAX(result_id) result_id, business_date 
         FROM md_result 
         WHERE business_date BETWEEN TO_DATE('{from_date}', 'YYYY-MM-DD') AND TO_DATE('{to_date}', 'YYYY-MM-DD') 
         GROUP BY business_date) r
    INNER JOIN md_result_details d ON d.result_id = r.result_id
    INNER JOIN md_security s ON s.md_security_id = d.md_security_id
    INNER JOIN md_sec_addin a ON a.md_security_id = s.md_security_id AND a.result_id = r.result_id
    LEFT JOIN idc_sec_ind_map i ON i.sub_industry_id = s.industry_cd
    INNER JOIN country c ON c.country_cd = s.country_cd
    """
    try:
        return pd.read_sql_query(sql, conn)
    except Exception as e:
        print(f"Error executing SQL query: {e}")
        return pd.DataFrame()

conn = get_db_connection()
if conn:
    df = fetch_data(conn)
    conn.close()
    
    # Save raw data
    yesterday = datetime.date.today() - datetime.timedelta(days=1)
    csv_path = os.path.join(save_dir, f"raw_db_pull_{yesterday.strftime('%Y-%m-%d')}.csv")
    df.to_csv(csv_path, index=False)
    print(f"Raw data saved to {csv_path}")
else:
    df = pd.DataFrame() # Ensure df exists

# --- 4. Data Processing & Analysis ---
def process_data(df):
    """Processes the raw data to calculate WoW changes and filter tables."""
    if df.empty:
        return pd.DataFrame(), pd.DataFrame()

    df['business_date'] = pd.to_datetime(df['business_date'])
    df = df.sort_values(['ticker', 'business_date'])

    # Calculate WoW changes
    df['fee_wow'] = df.groupby('ticker')['fee_bps'].diff(periods=5)
    df['price_wow_pct'] = df.groupby('ticker')['price'].pct_change(periods=5) * 100
    df['short_squeeze_score_wow'] = df.groupby('ticker')['short_squeeze_score'].diff(periods=5)

    latest_df = df[df['business_date'] == df['business_date'].max()].copy()

    # Filter for specials table
    specials = latest_df[latest_df['fee_wow'] > 0].sort_values(by='fee_bps', ascending=False).head(5)
    
    # Filter for squeeze table
    squeeze = latest_df[(latest_df['short_squeeze_score'] > 60) &
                        (latest_df['on_loan_value'] > 1_000_000) &
                        (latest_df['borrower_count'] >= 2)].sort_values(by='short_squeeze_score', ascending=False).head(5)
    
    return specials, squeeze

filtered_specials, filtered_squeeze = process_data(df)

# --- 5. HTML Report Generation ---
def generate_html_report(market_notes, headlines, specials, squeeze, takeaways):
    """Generates and saves the HTML report."""
    date_str = (datetime.date.today() - datetime.timedelta(days=1)).strftime('%B %d, %Y')
    
    # Helper for table rows
    def create_specials_rows(df):
        rows = ""
        for _, row in df.iterrows():
            rows += f"""<tr>
                <td>{row['ticker']}</td>
                <td>{row['name']}</td>
                <td>{row['industry']}</td>
                <td>{row['fee_bps']:.0f} BPS</td>
                <td>+{row['fee_wow']:.0f} BPS</td>
            </tr>"""
        return rows

    def create_squeeze_rows(df):
        rows = ""
        for _, row in df.iterrows():
            rows += f"""<tr>
                <td>{row['ticker']}</td>
                <td>{row['name']}</td>
                <td>{row['industry']}</td>
                <td>${row['price']:.2f}</td>
                <td>{row['price_wow_pct']:.1f}%</td>
                <td>{row['short_squeeze_score']:.2f}</td>
                <td>{row['short_squeeze_score_wow']:+.2f}</td>
            </tr>"""
        return rows

    html_template = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>EquiLend D&A Daily Digest</title>
        <style>
            body {{ font-family: Arial, sans-serif; color: #333; background-color: #f4f4f4; margin: 20px; }}
            h1, h2 {{ color: #003366; }}
            table {{ width: 100%; border-collapse: collapse; margin-top: 10px; }}
            th, td {{ border: 1px solid #808080; padding: 8px; text-align: left; }}
            th {{ background-color: #003366; color: white; }}
        </style>
    </head>
    <body>
        <h1>EquiLend D&A Daily Digest</h1>
        <p style="text-align: center;">{date_str}</p>
        
        <h2>📈 Market Notes</h2>
        <ul>{''.join([f'<li>{note}</li>' for note in market_notes])}</ul>
        
        <h2>🔥 Major Headlines & What Our Data Shows</h2>
        {''.join([f"<p><strong>{hl['title']}</strong><br>{hl['description']}<br><strong>Data Shows:</strong> {hl['data_shows']}</p>" for hl in headlines])}
        
        <h2>📊 Today's Specials & Hard-to-Borrow</h2>
        <table>
            <thead><tr><th>Ticker</th><th>Name</th><th>Industry</th><th>Fee (BPS)</th><th>Momentum (WoW)</th></tr></thead>
            <tbody>{create_specials_rows(specials)}</tbody>
        </table>
        <br>
        <table>
            <thead><tr><th>Ticker</th><th>Company Name</th><th>Industry</th><th>Price</th><th>Price (WoW)</th><th>Short Squeeze Score</th><th>Score (WoW)</th></tr></thead>
            <tbody>{create_squeeze_rows(squeeze)}</tbody>
        </table>
        
        <h2>💡 Key Takeaways</h2>
        <ul>{''.join([f'<li>{kt}</li>' for kt in takeaways])}</ul>
        
        <p style="text-align: right; font-style: italic; color: #808080;">Source: EquiLend Data & Analytics</p>
    </body>
    </html>
    """
    
    yesterday = datetime.date.today() - datetime.timedelta(days=1)
    html_path = os.path.join(save_dir, f"equilend_daily_digest_{yesterday.strftime('%Y-%m-%d')}.html")
    with open(html_path, 'w') as f:
        f.write(html_template)
    print(f"HTML report saved to {html_path}")
    return html_path, html_template

html_path, html_template = generate_html_report(market_notes, headlines, filtered_specials, filtered_squeeze, key_takeaways)

Error reading Word document 'daily_content.docx': Package not found at 'daily_content.docx'
Error: Database credentials not found in .env file.
HTML report saved to /Users/bob/equilend_daily/equilend_daily_digest_2025-07-12.html


### Sync Clouds & Send Email

In [ ]:
# --- 6. Cloud Sync & Email ---
def sync_to_cloud(file_path, remotes):
    """Syncs a file to configured cloud remotes using rclone."""
    for remote in remotes:
        try:
            subprocess.run(['rclone', 'copy', file_path, remote], check=True, capture_output=True, text=True)
            print(f"Synced {os.path.basename(file_path)} to {remote}")
        except FileNotFoundError:
            print("Error: 'rclone' command not found. Please install and configure rclone.")
            break
        except subprocess.CalledProcessError as e:
            print(f"Error syncing to {remote}: {e.stderr}")

def send_email_draft(html_content):
    """Sends the HTML report as an email draft."""
    sender = os.getenv('EMAIL_SENDER')
    password = os.getenv('EMAIL_PASSWORD')
    recipient = os.getenv('EMAIL_RECIPIENT')
    smtp_server = os.getenv('SMTP_SERVER', 'smtp.office365.com')
    smtp_port = int(os.getenv('SMTP_PORT', 587))

    if not all([sender, password, recipient]):
        print("Error: Email credentials not found in .env file.")
        return

    msg = MIMEMultipart()
    msg['From'] = sender
    msg['To'] = recipient
    msg['Subject'] = f"Draft: EquiLend D&A Daily Digest - {(datetime.date.today() - datetime.timedelta(days=1)).strftime('%B %d, %Y')}"
    msg.attach(MIMEText(html_content, 'html'))

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender, password)
            server.send_message(msg)
        print("Email draft sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")

# Sync files to cloud
sync_to_cloud(csv_path, cloud_remotes)
sync_to_cloud(html_path, cloud_remotes)

# Send email
send_email_draft(html_template)